# This notebook shows a proof of concept for a post-processing modification to the MTNA-s system.
This is quite an unstable proposition, howver, since the output of the NN is not consistent.

In [8]:
import pickle
import re
import pandas as pd
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros

#  start here


In [4]:
df_test = pickle.load(open('gold_for_bi_lstm.pkl', 'rb'))
df_train = pickle.load(open('train_for_bi_lstm.pkl', 'rb'))

# step 1

In [5]:
# define documents - these are lemmatised texts in df_train and df_gold
docs = df_train.lemmatised.tolist()
test_docs = df_test.lemmatised.tolist()
# finds the max sentence length
len(max(docs, key = lambda i: len(i)))

79

In [7]:
# tokenize texts
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
encoded_test_docs = t.texts_to_sequences(test_docs)
# print(encoded_docs)
# pad documents to a max length of 80 words
max_length = 79
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post', truncating = 'post')
padded_test_docs = pad_sequences(encoded_test_docs, maxlen=max_length, padding='post', truncating = 'post')

In [13]:
path = 'E:/Study/Vu/Machine learning experiments/glove.6B/glove.6B.200d.txt'
# load the whole embedding into memory
embeddings_index = dict()
f = open(path, encoding = 'utf-8')
# f = open('/media/peter/BigExternal/Study/Vu/Machine learning experiments/glove.6B/glove.6B.200d.txt', encoding = 'utf-8')

for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print(f'Loaded {len(embeddings_index)} word vectors.')

Loaded 400000 word vectors.


In [15]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 200))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector    

#  5 classifiers - 1 per category

In [10]:
df_test = df_gold

In [17]:
df_test = df_test[['text', 'category0', 'category1','category2','category3']]

df_test['cat_ambience'] = 0
df_test['cat_anecdotes/miscellaneous'] = 0
df_test['cat_food'] = 0
df_test['cat_price'] = 0
df_test['cat_service'] = 0


df_test.loc[(df_test['category0'] == 'ambience') | (df_test['category1'] == 'ambience') | (df_test['category2'] == 'ambience')| (df_test['category3'] == 'ambience'),'cat_ambience'] = 1
df_test.loc[(df_test['category0'] == 'anecdotes/miscellaneous') | (df_test['category1'] == 'anecdotes/miscellaneous') | (df_test['category2'] == 'anecdotes/miscellaneous')| (df_test['category3'] == 'anecdotes/miscellaneous'),'cat_anecdotes/miscellaneous'] = 1
df_test.loc[(df_test['category0'] == 'food') | (df_test['category1'] == 'food') | (df_test['category2'] == 'food')| (df_test['category3'] == 'food'),'cat_food'] = 1
df_test.loc[(df_test['category0'] == 'price') | (df_test['category1'] == 'price') | (df_test['category2'] == 'price')| (df_test['category3'] == 'price'),'cat_price'] = 1
df_test.loc[(df_test['category0'] == 'service') | (df_test['category1'] == 'service') | (df_test['category2'] == 'service')| (df_test['category3'] == 'service'),'cat_service'] = 1

In [19]:
df_train = df_train[['text', 'category0', 'category1','category2','category3']]
df_train['cat_ambience'] = 0
df_train['cat_anecdotes/miscellaneous'] = 0
df_train['cat_food'] = 0
df_train['cat_price'] = 0

df_train['cat_service'] = 0
df_train.loc[(df_train['category0'] == 'ambience') | (df_train['category1'] == 'ambience') | (df_train['category2'] == 'ambience')| (df_train['category3'] == 'ambience'),'cat_ambience'] = 1
df_train.loc[(df_train['category0'] == 'anecdotes/miscellaneous') | (df_train['category1'] == 'anecdotes/miscellaneous') | (df_train['category2'] == 'anecdotes/miscellaneous')| (df_train['category3'] == 'anecdotes/miscellaneous'),'cat_anecdotes/miscellaneous'] = 1
df_train.loc[(df_train['category0'] == 'food') | (df_train['category1'] == 'food') | (df_train['category2'] == 'food')| (df_train['category3'] == 'food'),'cat_food'] = 1

df_train.loc[(df_train['category0'] == 'price') | (df_train['category1'] == 'price') | (df_train['category2'] == 'price')| (df_train['category3'] == 'price'),'cat_price'] = 1
df_train.loc[(df_train['category0'] == 'service') | (df_train['category1'] == 'service') | (df_train['category2'] == 'service')| (df_train['category3'] == 'service'),'cat_service'] = 1

# functional api

The main input will receive the headline, as a sequence of integers (each integer encodes a word). The integers will be between 1 and 10,000 (a vocabulary of 10,000 words) and the sequences will be 100 words long.

# without concatenated cnn - for compars (window = 5)

In [136]:
from keras.layers import Bidirectional, Input, Embedding, LSTM, Dense, merge, Conv1D, GlobalMaxPooling1D, Concatenate, concatenate

from keras.models import Model

main_input = Input(shape=(79,), name='main_input')

x = Embedding(output_dim=200, input_dim=vocab_size, input_length=79,weights=[embedding_matrix],trainable=True)(main_input)
lstm_out = Bidirectional(LSTM(units = 300, activation = 'tanh', dropout = 0.2, recurrent_dropout = 0.2, return_sequences = True))(x)

cnn5_in = Conv1D(filters = 100, kernel_size = 5, padding='same', activation='tanh',strides=1,input_shape = (None, 300))(lstm_out)
cnn5_max_out = GlobalMaxPooling1D()(cnn5_in)
bi_max_out = GlobalMaxPooling1D()(lstm_out)
merge_one = concatenate([bi_max_out, cnn5_max_out])
final_out = Dense(2, activation='softmax')(merge_one)
model = keras.Model(inputs=main_input, outputs=final_out)

model.compile(optimizer='Adadelta', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 79)           0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 79, 200)      756400      main_input[0][0]                 
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 79, 600)      1202400     embedding_4[0][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 79, 100)      300100      bidirectional_3[0][0]            
____________________________________________________________________________________________

# modified with post processing

In [137]:
model.fit(padded_docs, df_train['cat_ambience'].tolist(), epochs=20, validation_split=0.2, verbose = False)
output_ambience = model.predict(padded_test_docs)
system_predictions_amb = pd.DataFrame(np.argmax(output_ambience,axis=1), columns=['Predicted'])
print('ambience', '\n',classification_report(df_test['cat_ambience'].tolist(), system_predictions_amb))

c:\python\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


ambience 
               precision    recall  f1-score   support

           0       0.94      0.99      0.96       682
           1       0.91      0.61      0.73       118

    accuracy                           0.93       800
   macro avg       0.92      0.80      0.85       800
weighted avg       0.93      0.93      0.93       800



In [138]:
model.fit(padded_docs, df_train['cat_anecdotes/miscellaneous'].tolist(), epochs=20, validation_split=0.2, verbose = False)
output_anec = model.predict(padded_test_docs)
system_predictions_anec = pd.DataFrame(np.argmax(output_anec,axis=1), columns=['Predicted'])
print('anecdotes/miscellaneous', '\n',classification_report(df_test['cat_anecdotes/miscellaneous'].tolist(), system_predictions_anec))

anecdotes/miscellaneous 
               precision    recall  f1-score   support

           0       0.81      0.98      0.89       566
           1       0.90      0.46      0.61       234

    accuracy                           0.83       800
   macro avg       0.86      0.72      0.75       800
weighted avg       0.84      0.83      0.81       800



In [139]:
model.fit(padded_docs, df_train['cat_food'].tolist(), epochs=20, validation_split=0.2, verbose = False)
output_food = model.predict(padded_test_docs)
system_predictions_food = pd.DataFrame(np.argmax(output_food,axis=1), columns=['Predicted'])
print('food', '\n',classification_report(df_test['cat_food'].tolist(), system_predictions_food))

food 
               precision    recall  f1-score   support

           0       0.90      0.92      0.91       382
           1       0.93      0.91      0.92       418

    accuracy                           0.92       800
   macro avg       0.91      0.92      0.91       800
weighted avg       0.92      0.92      0.92       800



In [140]:
model.fit(padded_docs, df_train['cat_price'].tolist(), epochs=20, validation_split=0.2, verbose = False)
output_price = model.predict(padded_test_docs)
system_predictions_price = pd.DataFrame(np.argmax(output_price,axis=1), columns=['Predicted'])
print('price', '\n',classification_report(df_test['cat_price'].tolist(), system_predictions_price))

price 
               precision    recall  f1-score   support

           0       0.97      0.99      0.98       717
           1       0.94      0.71      0.81        83

    accuracy                           0.96       800
   macro avg       0.95      0.85      0.89       800
weighted avg       0.96      0.96      0.96       800



In [141]:
model.fit(padded_docs, df_train['cat_service'].tolist(), epochs=20, validation_split=0.2, verbose = False)
output_service = model.predict(padded_test_docs)
system_predictions_service = pd.DataFrame(np.argmax(output_service,axis=1), columns=['Predicted'])
print('service', '\n',classification_report(df_test['cat_service'].tolist(), system_predictions_service))

service 
               precision    recall  f1-score   support

           0       0.96      0.99      0.97       628
           1       0.95      0.84      0.89       172

    accuracy                           0.96       800
   macro avg       0.95      0.92      0.93       800
weighted avg       0.96      0.96      0.96       800



## list output where all categories = 0

In [142]:
df_test['pred_amb'] = list(system_predictions_amb.Predicted)
df_test['pred_anec']= list(system_predictions_anec.Predicted)
df_test['pred_food']= list(system_predictions_food.Predicted)
df_test['pred_price']= list(system_predictions_price.Predicted)
df_test['pred_service'] = list(system_predictions_service.Predicted)

In [143]:
df_nolabel = df_test.loc[(df_test.cat_ambience == 0) & (df_test.pred_anec == 0) & (df_test.pred_food == 0) & (df_test.pred_price == 0) & (df_test.pred_service == 0)]

In [144]:
nolabel_list = (list(df_nolabel.index.values))

In [145]:
proba_dict = {'ambience':output_ambience, 'anecdotes/miscellaneous':output_anec, 'food':output_food,
              'price':output_price,'service':output_service}

In [146]:
for key, srs in proba_dict.items():
    for idx in nolabel_list:
        for i, ls in enumerate(srs):                
            if i == int(idx) and ls[1]>0.4:
                name = key
                print ('predicted = :', name, '\t', '\t', '\t', 'actual = :', df_test[f'cat_{name}'].iloc[i])                                                                                        

predicted = : ambience 	 	 	 actual = : 0
predicted = : ambience 	 	 	 actual = : 0
predicted = : ambience 	 	 	 actual = : 0
predicted = : ambience 	 	 	 actual = : 0
predicted = : anecdotes/miscellaneous 	 	 	 actual = : 1
predicted = : anecdotes/miscellaneous 	 	 	 actual = : 1
predicted = : anecdotes/miscellaneous 	 	 	 actual = : 1
predicted = : food 	 	 	 actual = : 1
predicted = : service 	 	 	 actual = : 1
predicted = : service 	 	 	 actual = : 1
predicted = : service 	 	 	 actual = : 0
